In [1]:
#import necessary libraries
import geopandas as gpd

In [8]:
buildings = gpd.read_file('Data/Southampton.gpkg')
floodzone = gpd.read_file('Data/Flood_Map_for_Planning_Rivers_and_Sea_Flood_Zone_2Polygon.shp')

In [9]:
#Convert buildings crs to match floodzone crs
buildings= buildings.to_crs(floodzone.crs)
#Ensure both shapefiles are in the same crs
print(buildings.crs)
print(floodzone.crs)

EPSG:27700
EPSG:27700


In [10]:
#Perform a spatial join to find buildings that intersect with floodzone
buildings_in_floodzone = gpd.sjoin(
    buildings,
    floodzone,
    how = 'inner',
    predicate = 'intersects'
)
buildings_in_floodzone = buildings_in_floodzone[buildings.columns]
#Save to new file
buildings_in_floodzone.to_file('buildings_in_floodzone.shp')

In [11]:
import folium

In [13]:
buildings_in_floodzone = buildings_in_floodzone.to_crs('EPSG:4326')
floodzone = floodzone.to_crs('EPSG:4326')
bounds = buildings_in_floodzone.total_bounds
m = folium.Map(
    location=[(bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2],
    zoom_start=13,
    tiles='OpenStreetMap'
)
# Add flood zone layer
flood_layer = folium.GeoJson(
    floodzone,
    name='Flood Zone',
    style_function=lambda x: {
        'fillColor': '#9100cc',  # Purple
        'color': '#9100cc',
        'weight': 1,
        'fillOpacity': 0.4
    },
    tooltip=folium.GeoJsonTooltip(fields=['type'], aliases=['Flood Type:']), #customize tooltip to display popup of flood type
    show=True
).add_to(m)

# Add buildings in flood zone layer
buildings_layer = folium.GeoJson(
    buildings_in_floodzone,
    name='Buildings at Risk',
    style_function=lambda x: {
        'fillColor': '#FF4500',  # Orange-red
        'color': '#FF4500',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip( #customize popup to display fields
        fields=['id'],  # Display building ID
        aliases=['Building ID'],
        sticky=True  # Makes tooltip stay visible
    ),
    show=True
).add_to(m)

m.save('Flood_Risk_Map_osm.html')